# Correlación Estadísticamente Significativa: El Problema de las Múltiples Comparaciones 📊

---

## **# 1. De la Correlación Muestral ($r$) a la Correlación Poblacional ($\rho$)**

Cuando calculamos el **Coeficiente de Correlación de Pearson** ($r$) entre dos variables, solo estamos midiendo la relación en nuestra **muestra** de datos.

La pregunta clave que la estadística responde es: **¿Es esta correlación ($r$) lo suficientemente fuerte como para concluir que existe una correlación real ($\rho$) en la población, o podría ser simplemente el resultado del azar en nuestra muestra?**

### **Prueba de Hipótesis para Correlación**

Para determinar la significancia, usamos una prueba $t$ de hipótesis:

| Hipótesis | Significado | Objetivo |
| :--- | :--- | :--- |
| **Hipótesis Nula** ($H_0$) | **No hay correlación.** ($\rho = 0$) | Queremos **Rechazar** esta hipótesis. |
| **Hipótesis Alternativa** ($H_A$) | **Existe correlación.** ($\rho \ne 0$) | Queremos **Aceptar** esta hipótesis. |

El estadístico de prueba $t$ (si se cumplen ciertos supuestos, como la normalidad bivariada) se calcula como:

$$
t = r \sqrt{\frac{N-2}{1-r^2}}
$$

Donde:
* $r$: Coeficiente de correlación muestral.
* $N$: Número de observaciones.
* $t$: Estadístico de prueba que sigue una distribución $t$ de Student con $N-2$ grados de libertad.

Si el valor absoluto de $t$ es lo suficientemente grande (es decir, el $p$-valor es bajo), concluimos que la correlación es **estadísticamente significativa**.

---

## **# 2. El Problema en Ciencia de Datos: Múltiples Comparaciones**

En la práctica de la Ciencia de Datos, no analizamos solo un par de variables, sino una matriz de correlación completa (un **Mapa de Calor de Correlación**). Si tienes $n$ *features* en tu *dataset*, el número total de pares de correlaciones que pruebas es:

$$
\text{Número de Pruebas }(m) = \frac{n(n-1)}{2}
$$

### **Ejemplo del Riesgo**

Si tienes $n=20$ variables, realizas $m = 20(19)/2 = 190$ pruebas de correlación.

Si usas un $\alpha = 0.05$ (5%) sin corregir, la probabilidad de encontrar **al menos una correlación significativa por puro azar** (Falso Positivo) se dispara.

### **La Solución: Controlar el FDR (Benjamini-Hochberg)**

Para un trabajo robusto, no podemos usar el $p$-valor crudo. Debemos aplicar una corrección para el **Control del Error Tipo I**. El método recomendado es el de **Benjamini-Hochberg (BH)**, que controla la **Tasa de Descubrimiento Falso (FDR)**.

**Proceso de Corrección:**

1.  Calcula el $p$-valor para los 190 pares de correlación.
2.  Trata estos 190 $p$-valores como el conjunto completo de pruebas ($m=190$).
3.  Aplica el método BH para obtener los $p$-valores ajustados.
4.  Declara significativa una correlación solo si su **$p$-valor ajustado por BH** es menor que el $\text{FDR}$ deseado (ej. 0.05).

---

## **# 3. Importancia en Ciencia de Datos**

La matriz de correlación ajustada estadísticamente es vital en varias etapas del *pipeline* de un modelo:

### **A. Detección de Multicolinealidad**

* **Definición:** Ocurre cuando dos o más *features* predictoras están altamente correlacionadas entre sí.
* **Problema:** Si dos *features* están significativa y fuertemente correlacionadas, aportan la misma información al modelo. Esto infla la varianza de los coeficientes en modelos de regresión lineal (volviendo a los coeficientes inestables e ininterpretables).
* **Acción:** Si dos variables tienen una correlación fuerte **y estadísticamente significativa**, se debe eliminar una de ellas (o combinarlas).

### **B. Selección de Features (Feature Selection)**

* La matriz de correlación ajustada es una herramienta inicial para la selección de variables:
    * **Correlación Fuerte con la Variable Objetivo:** Indica que la variable es un buen predictor.
    * **Correlación Débil (No Significativa) con la Variable Objetivo:** Indica que la variable es ruido. Se puede eliminar de forma segura.

### **C. Modelos de Series de Tiempo (ARIMA/SARIMA)**

* Las funciones **ACF y PACF** (que veremos en el *notebook* ARIMA) son, en esencia, **correlaciones**. Si tienes un *pico* en ACF o PACF, necesitas saber si ese pico es **estadísticamente significativo** (es decir, el $p$-valor de la autocorrelación es bajo), lo que te indica qué parámetros usar en el modelo.

---

## **# 4. Implementación en Python (Ejemplo de $p$-valor)**

Usaremos `scipy.stats` para obtener la correlación y el $p$-valor de un par, y `statsmodels` para ilustrar la aplicación de la corrección a una lista de $p$-valores.

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests

# 1. Crear datos de ejemplo
np.random.seed(42)
N = 50 # Tamaño de la muestra
df = pd.DataFrame({
    'Variable_A': np.random.normal(0, 1, N),
    'Variable_B': np.random.normal(0, 1, N) + 0.5 * np.arange(N), # Correlación fuerte
    'Variable_C': np.random.normal(0, 1, N) # Correlación nula
})

# 2. Correlación y p-valor para cada par (N=3 variables, m=3 pruebas)
# CORRECCIÓN: Usar los nombres de columna EXACTOS
pruebas = [
    ('Variable_A', 'Variable_B'), 
    ('Variable_A', 'Variable_C'), 
    ('Variable_B', 'Variable_C')
]

p_valores_brutos = []
correlaciones = []

for var1, var2 in pruebas:
    # pearsonr devuelve r (correlación) y p (p-valor)
    r, p = pearsonr(df[var1], df[var2])
    correlaciones.append(r)
    p_valores_brutos.append(p)
    print(f"Par ({var1}-{var2}): r={r:.4f}, p={p:.4f} (Bruto)")


# 3. Aplicar Corrección de Benjamini-Hochberg (BH)
# método='fdr_bh' es el método de Benjamini-Hochberg para controlar el FDR
reject, p_valores_bh_ajustados, _, _ = multipletests(p_valores_brutos, alpha=0.05, method='fdr_bh')

print("\n--- Resultados de la Corrección (BH) ---")
print(f"Nivel FDR deseado: 0.05")

for i, (p_ajustado, r_val) in enumerate(zip(p_valores_bh_ajustados, correlaciones)):
    es_significativo = reject[i]
    conclusion = "Significativa (Rechaza H0)" if es_significativo else "NO Significativa (Acepta H0)"
    
    print(f"Par ({pruebas[i][0]}-{pruebas[i][1]}): r={r_val:.4f}, p_ajustado={p_ajustado:.4f}")
    print(f"  > Conclusión BH: {conclusion}")

Par (Variable_A-Variable_B): r=-0.1725, p=0.2311 (Bruto)
Par (Variable_A-Variable_C): r=-0.1256, p=0.3848 (Bruto)
Par (Variable_B-Variable_C): r=-0.0636, p=0.6606 (Bruto)

--- Resultados de la Corrección (BH) ---
Nivel FDR deseado: 0.05
Par (Variable_A-Variable_B): r=-0.1725, p_ajustado=0.5772
  > Conclusión BH: NO Significativa (Acepta H0)
Par (Variable_A-Variable_C): r=-0.1256, p_ajustado=0.5772
  > Conclusión BH: NO Significativa (Acepta H0)
Par (Variable_B-Variable_C): r=-0.0636, p_ajustado=0.6606
  > Conclusión BH: NO Significativa (Acepta H0)
